In [1]:
import json

## Q1

In [2]:
with open("senators.json","r",encoding='utf-8') as f:
    data = json.loads(f.read())

In [3]:
data.keys()

dict_keys(['meta', 'objects'])

In [4]:
len(data['objects'])

100

In [5]:
keys = []
consistent = True
for idx, obj in enumerate(data['objects']):
    keys.append(set(list(obj.keys())))
    if idx!=0:
        if len(keys[idx-1].difference(keys[idx]))>0:
            consistent = False
print(consistent)
        
                    

True


## Q2

After looking at data in the json viewer, there can be two tables:

1. Senators (containing information about current senators
2. Members, personal details such as name, etc, member_id or senator_id could be the foriegn key

## Q3

In [6]:
keys = []
consistent = True
for idx, obj in enumerate(data['objects']):
    keys.append(set(list(obj['person'].keys())))
    if idx!=0:
        if len(keys[idx-1].difference(keys[idx]))>0:
            consistent = False
print(consistent)

True


In [7]:
keys = []
consistent = True
for idx, obj in enumerate(data['objects']):
    keys.append(set(list(obj['extra'].keys())))
    if idx!=0:
        if len(keys[idx-1].difference(keys[idx]))>0:
            consistent = False
print(consistent)

False


## Q4

In [8]:
keys = []
cnt = 0
for obj in data['objects']:
    if cnt == 0:
        keys = set(list(obj['extra'].keys()))
    else:
        keys = keys.union(set(list(obj['extra'].keys())))
    cnt+=1
print(keys)

{'address', 'rss_url', 'office', 'contact_form', 'how', 'fax'}


## Q5

In [9]:
from sqlalchemy import Column, Integer, Numeric, String, DateTime, ForeignKey, Boolean, create_engine
from sqlalchemy.orm import Session
from sqlalchemy.ext.declarative import declarative_base

In [10]:
Base = declarative_base()

class Senators(Base):
    __tablename__ = "senators"
    
    senator_id = Column(Integer(), autoincrement = True, primary_key = True)
    current = Column(Boolean(), nullable = False)
    description = Column(String(),nullable = False)
    enddate = Column(DateTime(), nullable = False)
    party = Column(String(), nullable = False)
    phone = Column(String(), nullable = False)
    role_type = Column(String(), nullable = False)
    role_type_label = Column(String(), nullable = False)
    senator_class = Column(String(), nullable = False)
    senator_class_label = Column(String(), nullable = False)
    senator_rank = Column(String(), nullable = False)
    senator_rank_label = Column(String(), nullable = False)
    startdate = Column(DateTime(), nullable = False)
    state = Column(String(), nullable = False)
    title = Column(String(), nullable = False)
    title_long = Column(String(), nullable = False)
    website = Column(String(), nullable = False)
    rss_url = Column(String(), nullable = True)
    contact_form = Column(String(), nullable = True)
    fax = Column(String(), nullable = True)
    how = Column(String(), nullable = True)
    office = Column(String(), nullable = True)
    address = Column(String(), nullable = True)
    detail_id = Column(Integer(), ForeignKey('details.detail_id'))
    
class Details(Base):
    __tablename__ = 'details'
    
    detail_id = Column(Integer(), autoincrement = True, primary_key = True)
    bioguideid = Column(String(), nullable = False)
    birthday = Column(DateTime(), nullable = False)
    cspanid = Column(Numeric(), nullable = False)
    firstname = Column(String(), nullable = False)
    gender = Column(String(), nullable = False)
    gender_label = Column(String(), nullable = False)
    last_name = Column(String(), nullable = False)
    link = Column(String(), nullable = False)
    middlename = Column(String(), nullable = True)
    name = Column(String(), nullable = False)
    namemod = Column(String(), nullable = False)
    nickname = Column(String(), nullable = False)
    osid = Column(String(), nullable = False)
    pvsid = Column(String(), nullable = False)
    sortname = Column(String(), nullable = False)
    twitterid = Column(String(), nullable = False)
    youtubeid = Column(String(), nullable = False)
    
engine = create_engine('sqlite:///senators.db')
Base.metadata.create_all(engine)    

In [11]:
## insert data
with open("senators.json", "r", encoding="utf-8") as f:
    data = json.loads(f.read())

In [12]:
from datetime import datetime
session = Session(engine)
for idx,obj in enumerate(data['objects']):
    senator = Senators(current = obj['current'],
                      description = obj['description'],
                      enddate = datetime.fromisoformat(obj['enddate']),
                      party = obj['party'],
                      phone = obj['phone'],
                      role_type = obj['role_type'],
                      role_type_label = obj['role_type_label'],
                      senator_class = obj['senator_class'],
                      senator_class_label = obj['senator_class_label'],
                      senator_rank = obj['senator_rank'],
                      senator_rank_label = obj['senator_rank_label'],
                      startdate = datetime.fromisoformat(obj['startdate']),
                      state = obj['state'],
                      title = obj['title'],
                      title_long = obj['title_long'],
                      website = obj['website'],
                      rss_url = obj['extra'].get('rss_url'),
                      contact_form = obj['extra'].get('contact_form'),
                      fax = obj['extra'].get('fax'),
                      how = obj['extra'].get('how'),
                      office = obj['extra'].get('office'),
                      address = obj['extra'].get('address'),
                      detail_id = idx+1)
    session.add(senator)
    try:
        session.commit()
    except:
        session.rollback()
        
    detail = Details(bioguideid = obj['person']['bioguideid'],
                     birthday = datetime.fromisoformat(obj['person']['birthday']),
                     cspanid = obj['person']['cspanid'],
                     firstname = obj['person']['firstname'],
                     gender = obj['person']['gender'],
                     gender_label = obj['person']['gender_label'],
                     last_name = obj['person']['lastname'],
                     link = obj['person']['link'],
                     middlename = obj['person'].get('middlename'),
                     name = obj['person']['name'],
                     namemod = obj['person']['namemod'],
                     nickname = obj['person']['nickname'],
                     osid = obj['person']['osid'],
                     pvsid = obj['person']['pvsid'],
                     sortname = obj['person']['sortname'],
                     twitterid = obj['person']['twitterid'],
                     youtubeid = obj['person']['youtubeid'])
    session.add(detail)
    try:
        session.commit()
    except:
        session.rollback()                     